## 5.3 Starvation
A process or thread is perpetually denied the resource it needs. In a simple scenario with two equal threads competing for execution time, starvation probably isn't a concern. Both threads should get plenty of chances to acquire the resource. However, if two threads are given different priorities, then that may not be the case. How different thread priorities get treated will depend on the operating system. But generally higher priority threads will be scheduled to execute more often, and that can leave low priority threads feeling hungry. Another thing that can lead to starvation is having too many concurrent threads.

To demonstrate thread starvation in Python, we'll modify the dining philosophers example by adding a local variable within the `philosopher` function to keep track of how many pieces of sushi each philosopher thread gets to eat.

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 5000

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    sushi_eaten = 0
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    sushi_eaten += 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
    print(name, 'took', sushi_eaten, 'pieces')

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_c)).start()


When it finishes, we see that each of the philosophers got a different amount of sushi and it's not particularly fair. Olivia took way more sushi than Barron or Steve. She has over 4000 of the pieces. But it's not because she's greedy. In this case, it's because of the order in which the 3 philosophers are taking chopsticks. Barron and Steve are both competing for `chopostick_a` as their first chopstick, but Olivia is the only philosopher going for `chopstick_b` first. Since she has less competition to get her first chopstick, she ends up getting to take sushi more frequently.

To make things fair for this example, we'll make all of the philosophers acquire the same two chopsticks, `chopostick_a` first and then `chopstick_b`.

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 5000

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    sushi_eaten = 0
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    sushi_eaten += 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
    print(name, 'took', sushi_eaten, 'pieces')

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_b)).start()


Let's see what happens if we drastically increase the number of philosophers at the dinner party using a `for` loop to create 50 instances of Barron, Olivia, and Steve. In this case, we'll have 150 philosophers competing for 5000 pieces of sushi.

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 5000

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    sushi_eaten = 0
    while sushi_count > 0: # eat sushi until it's all gone
        with first_chopstick:
            with second_chopstick:
                if sushi_count > 0:
                    sushi_count -= 1
                    sushi_eaten += 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
    print(name, 'took', sushi_eaten, 'pieces')

if __name__ == '__main__':
    for thread in range(50):
        threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
        threading.Thread(target=philosopher, args=('Olivia', chopstick_a, chopstick_b)).start()
        threading.Thread(target=philosopher, args=('Steve', chopstick_a, chopstick_b)).start()


We can see that some philosophers got a lot of sushi compared to others. Many of these philosophers starve completely taking 0. All of these threads were created with the same default priority, but because there're so many, some threads never got a chance to execute. This represents a real problem in programs containing a large number of threads. For example, if instead of feeding sushi to a bunch of philosophers, this program was a web server that created new threads to handle a huge number of incoming requests, some of those requests may never get processed that would lead to some very impatient and angry users on the other end. Some techniques can be used to improve or even guarantee fairness among threads, but that type of workload management is very situation dependent.